##### Implementation of MoCov3 to detect dog emotions.
- Batch Size : 128
- Image resoltion : 224 X 224
- Status : Building

In [2]:
import os
import math
import shutil
import random
import time

In [3]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [4]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as torchvision_models
from torch.utils.tensorboard import SummaryWriter

In [5]:
import moco.builder
import moco.loader
import moco.LARS

In [26]:
BATCH_SIZE = 128
RESOLUTION = 224
DISTRIBUTED = False
NUM_WORKERS = 4

##### Data Generator.
- Transforms yanked from the BYOL paper https://arxiv.org/abs/2006.07733.

In [28]:
augmentation1 = [
        transforms.RandomResizedCrop(RESOLUTION, scale=(0.8, 1.)),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.2, 0.1)  # not strengthened
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomApply([moco.loader.GaussianBlur([.1, 2.])], p=1.0),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]

augmentation2 = [
        transforms.RandomResizedCrop(RESOLUTION, scale=(0.8, 1.)),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.2, 0.1)  # not strengthened
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomApply([moco.loader.GaussianBlur([.1, 2.])], p=0.1),
        transforms.RandomApply([moco.loader.Solarize()], p=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.473, 0.440, 0.386],
                                     std=[0.269, 0.260, 0.262])
    ]

In [29]:
traindir = 'data/Daniel_Shan_Balico/train/'
testdir = 'data/Daniel_Shan_Balico/test/'
valdir = 'data/Daniel_Shan_Balico/val/'

train_dataset = datasets.ImageFolder(
        traindir,
        moco.loader.TwoCropsTransform(transforms.Compose(augmentation1), 
                                      transforms.Compose(augmentation2)))

##### ! Issue Faced ! If DISTRIBUTED = True, the following error occurs:  
- Default process group has not been initialized, please make sure to call init_process_group.
- SOLVED : Benched relevant code. (Would only need for multiple GPUs)

In [32]:
# if DISTRIBUTED:
#     train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
# else:
#     train_sampler = None

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle = False,
        num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)

In [37]:
train_l = enumerate(train_loader)
a = next(train_l)

In [41]:
type(a)
a[1]

[[tensor([[[[-2.0837, -2.0837, -2.1008,  ..., -2.1179, -2.1179, -2.1179],
            [-2.0837, -2.0837, -2.1008,  ..., -2.1008, -2.1008, -2.1179],
            [-2.0837, -2.0837, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
            ...,
            [-2.1008, -2.1008, -2.1008,  ..., -2.1179, -2.1008, -2.1008],
            [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
            [-2.1008, -2.0837, -2.0837,  ..., -2.1008, -2.1008, -2.1008]],
  
           [[-2.0007, -2.0007, -2.0182,  ..., -2.0357, -2.0357, -2.0357],
            [-2.0007, -2.0007, -2.0182,  ..., -2.0182, -2.0182, -2.0357],
            [-2.0007, -2.0007, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
            ...,
            [-2.0182, -2.0182, -2.0182,  ..., -2.0357, -2.0182, -2.0182],
            [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
            [-2.0182, -2.0007, -2.0007,  ..., -2.0182, -2.0182, -2.0182]],
  
           [[-1.7696, -1.7696, -1.7870,  ..., -1.8044, -1.8044, -1.804